In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from losses_metrics import *
from common import *

In [2]:
train = pd.read_csv(f'data/train.csv')
y = train[TARGETS].values
ix = np.where(train.groupby("question_body")["host"].transform("count") == 1)[0]

In [3]:
oofs2 = pd.read_csv('oofs/siamese_bert_2_oofs.csv', index_col=0).values
oofs3 = pd.read_csv('oofs/siamese_bert_3_oofs.csv', index_col=0).values
oofs4 = pd.read_csv('oofs/siamese_bert_4_oofs.csv', index_col=0).values
oofs5 = pd.read_csv('oofs/siamese_bert_5_oofs.csv', index_col=0).values

In [8]:
spearmanr_np(optimized_ahmet_round(oofs5, y, verbose=False), y)

/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


0.45104065459814635

In [53]:
from sklearn.model_selection import GroupKFold
from sklearn.tree import DecisionTreeRegressor

In [59]:
for t in TARGETS: print(t, train.groupby(t)['qa_id'].count())

question_asker_intent_understanding question_asker_intent_understanding
0.333333      68
0.444444      12
0.500000      33
0.555556      60
0.666667     503
0.777778     883
0.833333     108
0.888889    1579
1.000000    2833
Name: qa_id, dtype: int64
question_body_critical question_body_critical
0.333333    1506
0.444444     893
0.500000      94
0.555556     932
0.666667     845
0.777778     701
0.833333      55
0.888889     470
1.000000     583
Name: qa_id, dtype: int64
question_conversational question_conversational
0.000000    5409
0.333333     384
0.500000      30
0.666667     152
1.000000     104
Name: qa_id, dtype: int64
question_expect_short_answer question_expect_short_answer
0.000000     806
0.333333     522
0.500000     442
0.666667    1373
1.000000    2936
Name: qa_id, dtype: int64
question_fact_seeking question_fact_seeking
0.000000     315
0.333333     877
0.500000      81
0.666667    1326
1.000000    3480
Name: qa_id, dtype: int64
question_has_commonly_accepted_answer que

In [89]:
def DT_stack(oofs, y):
    new_oofs = oofs.copy()
    ds = []
    folds = list(GroupKFold(n_splits=10).split(X=train['question_body'], groups=train['question_body']))
    for i in range(N_TARGETS):
        max_depths = [2, 3, 4, 5, 6]
        best_d, best_score = 0, spearmanr(oofs[:,i], y[:,i])[0]
        for d in max_depths:
            regressor = DecisionTreeRegressor(random_state=0, max_depth=d, criterion='mae')
            pred = np.zeros(len(oofs))
            for fold_id, (train_index, valid_index) in enumerate(folds):
                regressor.fit(oofs[train_index,i].reshape(-1,1), y[train_index,i].reshape(-1,1))
                pred[valid_index] = regressor.predict(oofs[valid_index,i].reshape(-1,1))
            score = spearmanr(pred, y[:,i])[0]
            if score > best_score:
                print(i, d, np.round(score, 4))
                best_d = d
                best_score = score
                new_oofs[:, i] = pred
        ds.append(best_d)
        
    print('Optimal depths: ', ds)
    print('OOF score: ', spearmanr_np(new_oofs, y))
    print('OOF rounding score: ', spearmanr_np(optimized_ahmet_round(new_oofs, y, verbose=False), y))
    
    for i in range(N_TARGETS):
        if ds[i] > 0:
            regressor = DecisionTreeRegressor(random_state=0, max_depth=ds[i])
            regressor.fit(oofs[:,i].reshape(-1,1), y[:,i].reshape(-1,1))
            new_oofs[:, i] = regressor.predict(oofs5[:,i].reshape(-1,1))

    print('OOF score full sample fit: ', spearmanr_np(new_oofs, y))
    print('OOF rounding score full sample fit: ',spearmanr_np(optimized_ahmet_round(new_oofs,y,verbose=False),y))
    
    return new_oofs, ds

In [90]:
new_oofs5, ds = DT_stack(oofs5, y)
get_cvs(new_oofs5, y, ix)

5 2 0.4684


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/nump

12 2 0.4792
12 3 0.4849
14 2 0.5478
15 2 0.4899
15 4 0.4923
15 6 0.5002
19 2 0.4012
19 4 0.4013
19 6 0.4257
Optimal depths:  [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 3, 0, 2, 6, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
OOF score:  0.43430847931650507
OOF rounding score:  0.4408129853078696
OOF score full sample fit:  0.42673850094097193
OOF rounding score full sample fit:  0.463899228109432


,SpearmanR
CV,0.426739
CV unique rows,0.389399
CV ignoring hard targets,0.444832
CV unique rows ignoring hard targets,0.405319
CV w rounding,0.463899
CV unique rows w rounding,0.437469
CV ignoring hard targets w rounding,0.460360
CV unique rows ignoring hard targets w rounding,0.427677


In [92]:
new_oofs2, ds = DT_stack(oofs2, y)
get_cvs(new_oofs2, y, ix)

5 2 0.4576


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/nump

11 3 0.742
12 2 0.5177
12 3 0.5196
14 2 0.5896
15 2 0.5076
19 2 0.3005
Optimal depths:  [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 3, 0, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
OOF score:  0.4321603494226857
OOF rounding score:  0.43943027922970207
OOF score full sample fit:  0.4277694019295242
OOF rounding score full sample fit:  0.4463324469924288


,SpearmanR
CV,0.427769
CV unique rows,0.397140
CV ignoring hard targets,0.447186
CV unique rows ignoring hard targets,0.409876
CV w rounding,0.446332
CV unique rows w rounding,0.421152
CV ignoring hard targets w rounding,0.457175
CV unique rows ignoring hard targets w rounding,0.423829


In [93]:
new_oofs3
, ds = DT_stack(oofs3, y)
get_cvs(new_oofs3, y, ix)

2 4 0.4169
5 2 0.468


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/nump

12 2 0.4897
12 4 0.5008
14 2 0.5627
15 2 0.5308


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/nump

Optimal depths:  [0, 0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
OOF score:  0.4261788762822302
OOF rounding score:  0.4388476356721695
OOF score full sample fit:  0.42340967828881787
OOF rounding score full sample fit:  0.44283786966315486


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


,SpearmanR
CV,0.423410
CV unique rows,0.388269
CV ignoring hard targets,0.447859
CV unique rows ignoring hard targets,0.410042
CV w rounding,0.442838
CV unique rows w rounding,0.430067
CV ignoring hard targets w rounding,0.457755
CV unique rows ignoring hard targets w rounding,0.429109


In [94]:
new_oofs5_, ds = DT_stack(optimized_ahmet_roundhmet_round(oofs5, y, verbose=False), y)
get_cvs(new_oofs5_, y, ix)

/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


5 2 0.4748
Optimal depths:  [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
OOF score:  0.4516496136421249


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


OOF rounding score:  0.4518667225802349
OOF score full sample fit:  0.4514214279455092


/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


OOF rounding score full sample fit:  0.4516385368836192


,SpearmanR
CV,0.451421
CV unique rows,0.426824
CV ignoring hard targets,0.458415
CV unique rows ignoring hard targets,0.422834
CV w rounding,0.451639
CV unique rows w rounding,0.429754
CV ignoring hard targets w rounding,0.458647
CV unique rows ignoring hard targets w rounding,0.425974


In [82]:
new_oofs5_ = oofs5.copy()
for i in range(N_TARGETS):
    if ds[i] > 0:
        regressor = DecisionTreeRegressor(random_state=0, max_depth=ds[i])
        regressor.fit(oofs5[:,i].reshape(-1,1), y[:,i].reshape(-1,1))
        new_oofs5_[:, i] = regressor.predict(oofs5[:,i].reshape(-1,1))

spearmanr_np(new_oofs5_, y), spearmanr_np(optimized_ahmet_round(new_oofs5_, y, verbose=False), y)

(0.42673850094097193, 0.463899228109432)

In [83]:
get_cvs(new_oofs5_, y, ix)

,SpearmanR
CV,0.426739
CV unique rows,0.389399
CV ignoring hard targets,0.444832
CV unique rows ignoring hard targets,0.405319
CV w rounding,0.463899
CV unique rows w rounding,0.437469
CV ignoring hard targets w rounding,0.460360
CV unique rows ignoring hard targets w rounding,0.427677


In [84]:
# from lightgbm import LGBMRegressor

In [85]:
# new_oofs5 = oofs5.copy()
# for i in range(N_TARGETS):
#     max_depths = [2, 3, 4, 5, 6, 7, 8]
#     best_d, best_score = 0, spearmanr(oofs5[:,i], y[:,i])[0]
#     for d in max_depths:
#         regressor = LGBMRegressor(random_state=0, max_depth=d, learning_rate=0.01, n_estimators=200)
#         pred = np.zeros(len(oofs5))
#         for fold_id, (train_index, valid_index) in enumerate(folds):
#             regressor.fit(oofs5[train_index,i].reshape(-1,1), y[train_index,i],
#                           eval_set=[(oofs5[valid_index,i].reshape(-1,1), y[valid_index,i])],
#                           eval_metric='l1', verbose=False,
#                           early_stopping_rounds=5)
#             pred[valid_index] = regressor.predict(oofs5[valid_index,i].reshape(-1,1), 
#                                                   num_iteration=regressor.best_iteration_)
#         score = spearmanr(pred, y[:,i])[0]
#         #print(i, d, np.round(score, 4), np.round(best_score, 4))
#         if score > best_score:
#             print(i, d, np.round(score, 4))
#             best_d = d
#             best_score = score
#             new_oofs5[:, i] = pred
    
# spearmanr_np(new_oofs5, y), spearmanr_np(optimized_ahmet_round(new_oofs5, y, verbose=False), y)

In [4]:
get_cvs(oofs2, y, ix)

/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/home/robin/anaconda3/envs/GoogleQuest/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


,SpearmanR
CV,0.407278
CV unique rows,0.371782
CV ignoring hard targets,0.431038
CV unique rows ignoring hard targets,0.392546
CV w rounding,0.448776
CV unique rows w rounding,0.432710
CV ignoring hard targets w rounding,0.457380
CV unique rows ignoring hard targets w rounding,0.423055


In [5]:
get_cvs(oofs3, y, ix)

,SpearmanR
CV,0.410318
CV unique rows,0.374227
CV ignoring hard targets,0.433832
CV unique rows ignoring hard targets,0.394997
CV w rounding,0.443676
CV unique rows w rounding,0.428869
CV ignoring hard targets w rounding,0.458653
CV unique rows ignoring hard targets w rounding,0.427826


In [6]:
get_cvs(oofs4, y, ix)

,SpearmanR
CV,0.410915
CV unique rows,0.374687
CV ignoring hard targets,0.434463
CV unique rows ignoring hard targets,0.395569
CV w rounding,0.449694
CV unique rows w rounding,0.421206
CV ignoring hard targets w rounding,0.460004
CV unique rows ignoring hard targets w rounding,0.425556


In [7]:
get_cvs(oofs5, y, ix)

,SpearmanR
CV,0.408816
CV unique rows,0.371531
CV ignoring hard targets,0.432205
CV unique rows ignoring hard targets,0.391879
CV w rounding,0.451041
CV unique rows w rounding,0.428981
CV ignoring hard targets w rounding,0.458007
CV unique rows ignoring hard targets w rounding,0.425146


In [8]:
get_cvs((oofs2+oofs3)/2, y, ix)

,SpearmanR
CV,0.412118
CV unique rows,0.375994
CV ignoring hard targets,0.435919
CV unique rows ignoring hard targets,0.396939
CV w rounding,0.447414
CV unique rows w rounding,0.432944
CV ignoring hard targets w rounding,0.461039
CV unique rows ignoring hard targets w rounding,0.427675


In [9]:
get_cvs((oofs3+oofs5)/2, y, ix)

,SpearmanR
CV,0.413033
CV unique rows,0.376192
CV ignoring hard targets,0.436685
CV unique rows ignoring hard targets,0.396945
CV w rounding,0.453826
CV unique rows w rounding,0.433865
CV ignoring hard targets w rounding,0.462182
CV unique rows ignoring hard targets w rounding,0.428614


In [14]:
get_cvs((oofs2+oofs5)/2, y, ix)

,SpearmanR
CV,0.412006
CV unique rows,0.375443
CV ignoring hard targets,0.435828
CV unique rows ignoring hard targets,0.396227
CV w rounding,0.453693
CV unique rows w rounding,0.431880
CV ignoring hard targets w rounding,0.462048
CV unique rows ignoring hard targets w rounding,0.428392


In [12]:
get_cvs((oofs2+oofs3+oofs4+oofs5)/4, y, ix)

,SpearmanR
CV,0.414650
CV unique rows,0.378031
CV ignoring hard targets,0.438476
CV unique rows ignoring hard targets,0.399053
CV w rounding,0.455960
CV unique rows w rounding,0.435108
CV ignoring hard targets w rounding,0.464166
CV unique rows ignoring hard targets w rounding,0.429039
